In [19]:
import pandas as pd
import os
import datetime as dt

# Popular Music Arist Data
In this notebook, we collect data on popular music artists from various sources and clean and transform the data
- Transformation process includes narrowing the extracted data sets, dropping columns with data we're not interested in, and merging the data sets
- Three merged dataframes were created as final projects: 
    - Artists and Popularity Measures
    - Artist and Titles
    - Artists and Genres

In the final step, these dataframes are converted to dictionaries and inserted into database (MongoDB) 

## Data - Music Artist Popularity
source - https://www.kaggle.com/pieca111/music-artists-popularity

In [20]:
# Read the artists csv file in pandas; convert to dataframe

csv_file = "artists.csv"

artists_df = pd.read_csv(csv_file, low_memory=False)
artists_df = artists_df.drop(columns=['mbid', 'country_lastfm', 'artist_lastfm', 'tags_lastfm','ambiguous_artist'])

# new dataframe with US-only artists

us_df = artists_df.loc[artists_df['country_mb'] == "United States"].dropna()


pop_us_artists = us_df.rename(columns={"artist_mb" : "Artist",
                                               "tags_mb" : "Genre Tags", 
                                              "listeners_lastfm" : "Listeners_lfm",
                                               "scrobbles_lastfm": "Scrobbles_lfm"})

# pop_us_artists = pop_us_artists.sort_values('listeners_lastfm', ascending=False)
# pop_us_artists = pop_us_artists.drop_duplicates()pop_us_artists.drop(column=["country_mb"])
pop_us_artists = pop_us_artists.drop(columns=["country_mb"])
pop_us_artists.head()

,Artist,Genre Tags,Listeners_lfm,Scrobbles_lfm
2,Red Hot Chili Peppers,rock; alternative rock; 80s; 90s; rap; metal; ...,4620835.0,293784041.0
3,Rihanna,pop; dance; hip hop; reggae; contemporary r b;...,4558193.0,199248986.0
4,Eminem,turkish; rap; american; hip-hop; hip hop; hiph...,4517997.0,199507511.0
5,The Killers,synthpop; alternative rock; american; new wave...,4428868.0,208722092.0
6,Kanye West,synthpop; pop; american; hip-hop; hip hop; ele...,4390502.0,238603850.0


In [21]:
# new dataframe with artists and popularity measures 
artists = pop_us_artists.drop(columns=["Genre Tags"])
artists.head()

,Artist,Listeners_lfm,Scrobbles_lfm
2,Red Hot Chili Peppers,4620835.0,293784041.0
3,Rihanna,4558193.0,199248986.0
4,Eminem,4517997.0,199507511.0
5,The Killers,4428868.0,208722092.0
6,Kanye West,4390502.0,238603850.0


In [22]:
# new dataframe with artists and genre tags 
genre_tags = pop_us_artists.drop(columns=["Listeners_lfm" , "Scrobbles_lfm"])
genre_tags.head()

,Artist,Genre Tags
2,Red Hot Chili Peppers,rock; alternative rock; 80s; 90s; rap; metal; ...
3,Rihanna,pop; dance; hip hop; reggae; contemporary r b;...
4,Eminem,turkish; rap; american; hip-hop; hip hop; hiph...
5,The Killers,synthpop; alternative rock; american; new wave...
6,Kanye West,synthpop; pop; american; hip-hop; hip hop; ele...


In [23]:
# this is code splits genre_tag string into a list
# pop_us_artists["Genre Tags"] = pop_us_artists["Genre Tags"].str.split(";", n = -1)

## Data - Billboard-Weekly-Songs-With-Song-And-Artist-Spotify-Popularity
- source https://www.kaggle.com/miteshsingh/hollywood-music-dataset#Hollywood-Music-WCBS-Ranking.csv

In [24]:
# Reading billboard weekly CSV file into pandas; creating new dataframe
bb_as_file = "bb_artists_songs.csv"
bb_as_df = pd.read_csv(bb_as_file)
bb_as_df['Artist'].nunique()
print("There are " + str(len(bb_as_df)) + " rows and " + str(bb_as_df['Artist'].nunique()) + " unique artists in the dataset.")

There are 300600 rows and 9103 unique artists in the dataset.


In [25]:
# create new dataframe for 10 most recent years of data
bb_5 = bb_as_df.loc[bb_as_df["year"] >= 2002]
bb_5.head()
bb_5['Artist'].nunique()
# len(bb_5)
print("There are " + str(len(bb_5)) + " rows and " + str(bb_5['Artist'].nunique()) + " unique artists in the dataset.")

There are 76800 rows and 2864 unique artists in the dataset.


In [26]:
bb_5.head()

,date,Rank,Title,Artist,Weeks on chart,year,Spotify_Popularity,Artist_Popularity
223800,2002-01-01,1,How You Remind Me,Nickelback,18.0,2002,77,78
223801,2002-01-01,2,U Got It Bad,Usher,16.0,2002,69,82
223802,2002-01-01,3,Family Affair,Mary J. Blige,24.0,2002,69,72
223803,2002-01-01,4,Get The Party Started,P!nk,11.0,2002,62,83
223804,2002-01-01,5,Always On Time,Ja Rule Featuring Ashanti,8.0,2002,-1,-1


In [27]:
# remove columns and duplicate data; create a new dataframe of artists
bb_artists_pop = bb_5.drop(columns=["date", "Rank", "Weeks on chart", "year", "Title", "Spotify_Popularity"])
# bb_artists_pop = bb_artists_pop.sort_values("Artist_Popularity", ascending=False)
bb_artists_pop = bb_artists_pop.drop_duplicates().reset_index(drop=True)
# len(bb_artists_pop)
bb_artists_pop.head()

,Artist,Artist_Popularity
0,Nickelback,78
1,Usher,82
2,Mary J. Blige,72
3,P!nk,83
4,Ja Rule Featuring Ashanti,-1


In [28]:
# merge two artist dataframes into new dataframe
artist_merge = pd.merge(bb_artists_pop, artists, how='outer', on='Artist')
artist_merge.head()

,Artist,Artist_Popularity,Listeners_lfm,Scrobbles_lfm
0,Nickelback,78.0,NaN,NaN
1,Usher,82.0,2650020.0,39319140.0
2,Mary J. Blige,72.0,1309603.0,15708429.0
3,P!nk,83.0,2495211.0,68613195.0
4,Ja Rule Featuring Ashanti,-1.0,NaN,NaN


In [29]:
# Create a new data frame with artists and song titles
bb_artists_songs = bb_5.drop(columns=["date", "Rank", "Weeks on chart", "year", "Spotify_Popularity", "Artist_Popularity"])
bb_artists_songs = bb_artists_songs.drop_duplicates(subset="Title", keep='first').reset_index(drop=True)
bb_artists_songs.head()

,Title,Artist
0,How You Remind Me,Nickelback
1,U Got It Bad,Usher
2,Family Affair,Mary J. Blige
3,Get The Party Started,P!nk
4,Always On Time,Ja Rule Featuring Ashanti


## Data - Billboard-Yearly-Chart-With-Spotify-Popularity-Of-Song-And-Artist

In [30]:
bb_yearly = "billboard-yearly.csv"
bb_yearly_df = pd.read_csv(bb_yearly).sort_values('year')
bb_yearly_df.tail()

,year,Rank,Artist,Title,Spotify_Popularity,Artist_Popularity
5227,2012,28,Justin Bieber,Boyfriend,67,91
5226,2012,27,"Kanye West, Big Sean, Pusha T, 2 Chainz",Mercy,-1,-1
5224,2012,25,Jason Mraz,I Won’t Give Up,-1,81
5235,2012,36,Maroon 5 feat. Christina Aguilera,Moves Like Jagger,-1,-1
5299,2012,100,Linkin Park,Burn It Down,71,86


In [31]:
bb_recent_yearly = bb_yearly_df.loc[bb_yearly_df["year"] >= 2002]

bbry_artists = bb_recent_yearly.drop(columns=["year", "Rank", "Title", "Spotify_Popularity"]).drop_duplicates().reset_index(drop=True)
bbry_artists.head()

,Artist,Artist_Popularity
0,Brandy,67
1,City High feat. Eve,-1
2,No Doubt feat. Lady Saw,-1
3,OutKast feat. Killer Mike,-1
4,Angie Martinez feat. Lil’ Mo and Sacario,-1


In [32]:
artist_merge2 = pd.merge(artist_merge, bbry_artists, how='outer', on='Artist')
artist_merge2 = artist_merge2.drop(columns=["Artist_Popularity_y"])
artist_merge2 = artist_merge2.fillna(value=-1)
artist_merge2.head()
# len(artist_merge)
# artist_merge2.sort_values("Listeners_lfm")

,Artist,Artist_Popularity_x,Listeners_lfm,Scrobbles_lfm
0,Nickelback,78.0,-1.0,-1.0
1,Usher,82.0,2650020.0,39319140.0
2,Mary J. Blige,72.0,1309603.0,15708429.0
3,P!nk,83.0,2495211.0,68613195.0
4,Ja Rule Featuring Ashanti,-1.0,-1.0,-1.0


## Spotify 200 Charts
https://spotifycharts.com/regional/us/daily/latest

In [33]:
csv_file = "spot_2d.csv"

spot_200_df = pd.read_csv(csv_file, low_memory=False)
# spot_200_df.head()
spot_200_a = spot_200_df.drop(columns=["Track Name", "URL"]).rename(columns={"Position" : "Spot_200_19",
                                                                             "Streams" : "Spot_Streams_19"} )
spot_200_a = spot_200_a.drop_duplicates(subset='Artist', keep='first')
spot_200_a.head()

,Spot_200_19,Artist,Spot_Streams_19
0,1,Lil Uzi Vert,1394231
1,2,Arizona Zervas,1380549
2,3,Roddy Ricch,1291852
3,4,Post Malone,1202601
4,5,Harry Styles,1131215


In [34]:
artist_merge3 = pd.merge(artist_merge2, spot_200_a, how='outer', on='Artist')
artist_merge3 = artist_merge3.fillna(value=-1)
# artist_merge3 = artist_merge3.sort_values("Spotify_200", ascending=False)
artist_merge3 = artist_merge3.dropna().reset_index(drop=True)

artist_merge3.head()



,Artist,Artist_Popularity_x,Listeners_lfm,Scrobbles_lfm,Spot_200_19,Spot_Streams_19
0,Nickelback,78.0,-1.0,-1.0,-1.0,-1.0
1,Usher,82.0,2650020.0,39319140.0,-1.0,-1.0
2,Mary J. Blige,72.0,1309603.0,15708429.0,-1.0,-1.0
3,P!nk,83.0,2495211.0,68613195.0,-1.0,-1.0
4,Ja Rule Featuring Ashanti,-1.0,-1.0,-1.0,-1.0,-1.0


In [35]:
titles_merge = pd.merge(bb_artists_songs, spot_200_df, how='outer', on='Artist')
titles_merge = titles_merge.drop(columns=["Position", "Track Name", "Streams", "URL"]).dropna()

titles_merge.head()

,Title,Artist
0,How You Remind Me,Nickelback
1,Too Bad,Nickelback
2,Someday,Nickelback
3,Figured You Out,Nickelback
4,Feelin' Way Too Damn Good,Nickelback


## Spotify's Worldwide Daily Song Ranking
https://www.kaggle.com/edumucelli/spotifys-worldwide-daily-song-ranking

In [36]:
us_song = "US_daily_song_ranking.csv"
us_song_df = pd.read_csv(us_song)

grouped_song_df = us_song_df.groupby(['Artist'])['Title'].sum().to_frame('Title').reset_index()

grouped_US_df = us_song_df.groupby(['Artist'])['Streams'].sum().to_frame('Streams').reset_index()
grouped_US_df = grouped_US_df.rename(columns={'Streams':'Spot_stream_17'})
grouped_US_df.head()

,Artist,Spot_stream_17
0,*NSYNC,1081292
1,2 Chainz,192583965
2,21 Savage,481196247
3,22 Savage,5691910
4,2Pac,620891


In [37]:
artist_merge4 = pd.merge(artist_merge3, grouped_US_df, how='outer', on='Artist')
artist_merge4 = artist_merge4.fillna(value=-1)
artist_merge4 = artist_merge4.reset_index(drop=True)
artist_merge4.head()

,Artist,Artist_Popularity_x,Listeners_lfm,Scrobbles_lfm,Spot_200_19,Spot_Streams_19,Spot_stream_17
0,Nickelback,78.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,Usher,82.0,2650020.0,39319140.0,-1.0,-1.0,-1.0
2,Mary J. Blige,72.0,1309603.0,15708429.0,-1.0,-1.0,-1.0
3,P!nk,83.0,2495211.0,68613195.0,-1.0,-1.0,30836574.0
4,Ja Rule Featuring Ashanti,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [38]:
titles_merge2 = pd.merge(titles_merge, grouped_song_df, how='outer', on=['Title', 'Artist'])
titles_merge2 = titles_merge2.drop_duplicates()
titles_merge2.head()

,Title,Artist
0,How You Remind Me,Nickelback
1,Too Bad,Nickelback
2,Someday,Nickelback
3,Figured You Out,Nickelback
4,Feelin' Way Too Damn Good,Nickelback


## Trending YouTube Video Statistics and Comments
https://www.kaggle.com/datasnaek/youtube

In [39]:
yt_data = "youtube_artist_title_views.csv"
yt_data_df = pd.read_csv(yt_data)

yt_grouped_song_df = yt_data_df.groupby(['Artist'])['Title'].sum().to_frame('Title').reset_index()

grouped_yt_data_df = yt_data_df.groupby(['Artist'])['views'].sum().to_frame('views').reset_index()
grouped_yt_data_df = grouped_yt_data_df.rename(columns={'views':'YouTube_views'})
grouped_yt_data_df.head()


,Artist,YouTube_views
0,2CELLOS,432186
1,5 Seconds Of Summer,64307790
2,AMANDA PALMER,57481
3,ASKING ALEXANDRIA,12104824
4,AURORA,14469378


# Merged Artists Data Frame

In [40]:
artist_merge5 = pd.merge(artist_merge4, grouped_yt_data_df, how='outer', on='Artist')
artist_merge5 = artist_merge5.fillna(value=-1)
artist_merge5 = artist_merge5.rename(columns={"Artist_Popularity_x" : "Artist_Pop_Rating"} )
artist_merge5 = artist_merge5.reset_index(drop=True)
artist_merge5.head()

,Artist,Artist_Pop_Rating,Listeners_lfm,Scrobbles_lfm,Spot_200_19,Spot_Streams_19,Spot_stream_17,YouTube_views
0,Nickelback,78.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,Usher,82.0,2650020.0,39319140.0,-1.0,-1.0,-1.0,-1.0
2,Mary J. Blige,72.0,1309603.0,15708429.0,-1.0,-1.0,-1.0,-1.0
3,P!nk,83.0,2495211.0,68613195.0,-1.0,-1.0,30836574.0,-1.0
4,Ja Rule Featuring Ashanti,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


# Merged Titles Dataframe

In [41]:
titles_merge3 = pd.merge(titles_merge2, yt_grouped_song_df, how='outer', on=['Title', 'Artist'])
titles_merge3 = titles_merge3.drop_duplicates()
titles_merge3.head()

,Title,Artist
0,How You Remind Me,Nickelback
1,Too Bad,Nickelback
2,Someday,Nickelback
3,Figured You Out,Nickelback
4,Feelin' Way Too Damn Good,Nickelback


# Convert pandas dataframes to MongoDB collections

In [47]:
# Code for loading data from pandas dataframes into MongoDB
import json
import pymongo

genre = genre_tags.to_dict("records")

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["music_db"] # x stands for database name
mycol = mydb["artists_genres"] # y stands for collection/table name

music_db = mycol.insert_many(genre)

In [48]:
artists = artist_merge5.to_dict("records")

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["music_db"] # x stands for database name
mycol = mydb["artist_pop_measures"] # y stands for collection/table name

music_db = mycol.insert_many(artists)

In [49]:
titles = titles_merge3.to_dict("records")

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["music_db"] # x stands for database name
mycol = mydb["artist_titles"] # y stands for collection/table name

music_db = mycol.insert_many(titles)